In [1]:
import pandas as pd
from konlpy.tag import Mecab
tokenizer = Mecab()
csv_path = './news_crawler/news_data.csv'
csv_path2 = './news_crawler/news_data2.csv'


In [2]:
df1 = pd.read_table(csv_path, sep=',')
df1.head()

,news,code
0,파주시청. 사진제공=파주시 파주시청. 사진제공=파주시\n\n[파주=파이낸셜뉴스 강근...,사회
1,동영상 뉴스\n\n이천 물류창고 화재 발화지점으로 지목된 지하 2층에서 산소절단기의...,사회
2,황범순 의정부시 부시장 을지대학교 의정부캠퍼스 및 부속병원 공사현장 안전점검. 사진...,사회
3,귀갓길 여성을 쫓아가 성범죄를 시도한 20대 남성이 구속됐습니다.서울 강남경찰서는 ...,사회
4,(서울=연합뉴스) 대한약사회가 6일부터 코로나바이러스 감염증 대응 체계를 '사회적 ...,사회


In [3]:
df2 = pd.read_table(csv_path2, sep=',')
df2.head()

,news,code
0,기사 섹션 분류 안내\n\n기사의 섹션 정보는 해당 언론사의 분류를 따르고 있습니다...,경제
1,▶제21대 총선 실시간 개표 현황 및 결과 보기\n\n총선에서 여당이 다시 한 번 ...,경제
2,[뉴욕=AP/뉴시스]지난 10일 뉴욕 증권거래소 건물에 미국 국기가 게양되어 있다....,경제
3,"부산지역 주유소에서 판매하는 기름값이 휘발유는 평균 1200원대, 경유는 1000원...",경제
4,담배업계가 소비자의 취향을 저격한 다양한 담배 신제품들을 잇달아 선보이고 있다.이전...,경제


In [4]:
df=pd.concat([df1,df2],axis=0,ignore_index=True)

In [5]:
df.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
8822     True
8823     True
8824     True
8825     True
8826     True
Length: 8827, dtype: bool

In [6]:
df.describe()

,news,code
count,8827,8827
unique,6134,4
top,기사 섹션 분류 안내\n\n기사의 섹션 정보는 해당 언론사의 분류를 따르고 있습니다...,사회
freq,861,3000


In [7]:
df=df.drop_duplicates(['news'])
df.describe()

,news,code
count,6134,6134
unique,6134,4
top,[파이낸셜뉴스] 한국은행이 2008년 글로벌 금융위기 이후 처음으로 공개시장운영 증...,사회
freq,1,2226


In [8]:
len1 = len(df.loc[df['code']=='사회'])
len2 = len(df.loc[df['code']=='경제'])
len3 = len(df.loc[df['code']=='생활/문화'])
len4 = len(df.loc[df['code']=='IT/과학'])
min_len = min(len1,len2,len3,len4)

rate1 = (len1-min_len)/len1
rate2 = (len2-min_len)/len2
rate3 = (len3-min_len)/len3
rate4 = (len4-min_len)/len4

df=df.drop(df.loc[df['code']=='사회'].sample(frac=rate1).index)
df=df.drop(df.loc[df['code']=='경제'].sample(frac=rate2).index)
df=df.drop(df.loc[df['code']=='생활/문화'].sample(frac=rate3).index)
df=df.drop(df.loc[df['code']=='IT/과학'].sample(frac=rate4).index)
df

,news,code
0,파주시청. 사진제공=파주시 파주시청. 사진제공=파주시\n\n[파주=파이낸셜뉴스 강근...,사회
5,질서정연 코로나19 확산 방지를 위한 ‘물리적 거리 두기’가 ‘생활 속 거리 두기’...,사회
6,“코로나19에 걸렸다 나은 친구는 아무래도 좀 멀리하게 될 것 같아요. 재발 가능성...,사회
7,1977년 메이저리그 LA 다저스의 시즌 마지막 경기였다. 신인 타자 더스티 베이커...,사회
9,경찰 로고./뉴스1 © News1 신채린 기자 경찰 로고./뉴스1 © News1 신...,사회
...,...,...
8318,레고켐바이오사이언스(대표 김용주·사진)가 글로벌 기술수출에 또다시 성공했다. 201...,IT/과학
8319,제21대 국회의원선거 기간 중단됐던 네이버 실시간 급상승 검색어 서비스가 15일 오...,IT/과학
8320,아마존 [AFP=연합뉴스 자료사진] 아마존 [AFP=연합뉴스 자료사진]\n\n직원들...,IT/과학
8321,"제21대 국회의원선거가 임박한 가운데, 투표 및 개표 방송을 준비하는 기업들의 움직...",IT/과학


In [9]:
idx2word = {'101' : '경제', '102' : '사회', '103' : '생활/문화', '105' : 'IT/과학'}
labels = ['경제', '사회', '생활/문화','IT/과학']
stopwords = ['에','는','은','을','했','에게',
             '있','이','의','하','한','다',
             '과','때문','할','수','무단',
             '따른','및','금지','전재',
             '경향신문','기자','는데','가',
             '등','들','파이낸셜','저작','등','뉴스']

In [10]:
df['news'] = df['news'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
df['news']

0       파주시청 사진제공파주시 파주시청 사진제공파주시파주파이낸셜뉴스 강근주 기자 파주시는 ...
5       질서정연 코로나 확산 방지를 위한 물리적 거리 두기가 생활 속 거리 두기로 전환된 ...
6       코로나에 걸렸다 나은 친구는 아무래도 좀 멀리하게 될 것 같아요 재발 가능성 때문에...
7       년 메이저리그  다저스의 시즌 마지막 경기였다 신인 타자 더스티 베이커현 휴스턴 애...
9       경찰 로고뉴스   신채린 기자 경찰 로고뉴스   신채린 기자서울뉴스 이상학 기자  ...
                              ...                        
8318    레고켐바이오사이언스대표 김용주사진가 글로벌 기술수출에 또다시 성공했다 년 중국 포순...
8319    제대 국회의원선거 기간 중단됐던 네이버 실시간 급상승 검색어 서비스가 일 오후 시부...
8320    아마존 연합뉴스 자료사진 아마존 연합뉴스 자료사진직원들 코로나 확산 방지 조치 충분...
8321    제대 국회의원선거가 임박한 가운데 투표 및 개표 방송을 준비하는 기업들의 움직임이 ...
8322    롤 점검이 일 진행될 예정이다온라인게임 리그오브레전드이하 롤은 일 오전 시부터 오전...
Name: news, Length: 3608, dtype: object

In [11]:
texts=dict()
for label in labels:
    texts[label]=[]

context = df['news'].apply(lambda x: str(x).split())
for i,(words,label) in enumerate(zip(context,df['code'])):
    texts[label]+=words

    
counts=dict()
for label in labels:
    counts[label] = pd.Series(texts[label]).value_counts()
    print(label,counts[label])

경제 수           1407
등           1403
있다          1056
일           1024
코로나          971
            ... 
한국야쿠르트         1
상한을            1
선택이            1
검토한다서울시가       1
부리며            1
Length: 49777, dtype: int64
사회 일         1413
기자         975
및          865
있다         818
등          812
          ... 
예상됐다오후       1
체포시          1
산림치유는        1
유치한다또        1
오전에          1
Length: 50478, dtype: int64
생활/문화 수        1426
일        1126
등        1075
및         971
있다        895
         ... 
클레먼츠        1
진행자에게       1
올려두고        1
셋째로         1
서문에서        1
Length: 68735, dtype: int64
IT/과학 수         2105
등         1352
및         1268
일         1161
있는        1031
          ... 
바라보면서        1
말했다쏘카가       1
달이           1
동기인          1
오픈뱅킹         1
Length: 51909, dtype: int64


In [12]:
def duple_words(counts,stopwords):
    tmp_keys = list(counts.keys())
    c_keys=[]
    for k in tmp_keys:
        if len(counts[k]):
            c_keys.append(k)

    check = dict()
    duple= []
    for i,k in enumerate(c_keys):
        print(i)
        if i:
            for s in counts[k].keys():
                if s in check.keys() and s not in stopwords:
                   duple.append(s)
        else:
            for s in counts[k].keys():
                check[s]=1

    duple_dict = dict()
    for k in duple:
        duple_dict[k]=0
        if k in counts['사회'].keys():
            duple_dict[k]+=counts['사회'][k]
        if k in counts['IT/과학'].keys():
            duple_dict[k]+=counts['IT/과학'][k]
        if k in counts['생활/문화'].keys():
            duple_dict[k]+=counts['생활/문화'][k]

    duple_dict = sorted(duple_dict.items(),key=lambda x:-x[1])
    return duple_dict

duple_dict = duple_words(counts,stopwords)
print(duple_dict[:50])


0
1
2
3
[('일', 3700), ('있다', 2677), ('있는', 2331), ('년', 1990), ('월', 1749), ('코로나', 1745), ('통해', 1590), ('위해', 1561), ('것으로', 1505), ('재배포', 1360), ('위한', 1162), ('대한', 1098), ('지난', 1079), ('함께', 987), ('등을', 982), ('전', 958), ('무단전재', 863), ('제공', 857), ('대', 841), ('개', 762), ('서울', 758), ('국내', 756), ('중', 753), ('이번', 749), ('더', 748), ('오후', 728), ('다양한', 699), ('따라', 663), ('같은', 660), ('시', 643), ('도', 637), ('를', 630), ('따르면', 621), ('하는', 604), ('있도록', 599), ('가장', 587), ('지난해', 578), ('이후', 562), ('있다고', 552), ('온라인', 548), ('것이라고', 546), ('경우', 542), ('관련', 529), ('그', 516), ('올해', 509), ('관계자는', 502), ('하고', 500), ('다른', 496), ('많은', 489), ('이상', 487)]


In [13]:
duple_dict

[('일', 3700),
 ('있다', 2677),
 ('있는', 2331),
 ('년', 1990),
 ('월', 1749),
 ('코로나', 1745),
 ('통해', 1590),
 ('위해', 1561),
 ('것으로', 1505),
 ('재배포', 1360),
 ('위한', 1162),
 ('대한', 1098),
 ('지난', 1079),
 ('함께', 987),
 ('등을', 982),
 ('전', 958),
 ('무단전재', 863),
 ('제공', 857),
 ('대', 841),
 ('개', 762),
 ('서울', 758),
 ('국내', 756),
 ('중', 753),
 ('이번', 749),
 ('더', 748),
 ('오후', 728),
 ('다양한', 699),
 ('따라', 663),
 ('같은', 660),
 ('시', 643),
 ('도', 637),
 ('를', 630),
 ('따르면', 621),
 ('하는', 604),
 ('있도록', 599),
 ('가장', 587),
 ('지난해', 578),
 ('이후', 562),
 ('있다고', 552),
 ('온라인', 548),
 ('것이라고', 546),
 ('경우', 542),
 ('관련', 529),
 ('그', 516),
 ('올해', 509),
 ('관계자는', 502),
 ('하고', 500),
 ('다른', 496),
 ('많은', 489),
 ('이상', 487),
 ('오는', 486),
 ('총', 470),
 ('대해', 467),
 ('지난달', 461),
 ('새로운', 458),
 ('이날', 457),
 ('가운데', 449),
 ('큰', 447),
 ('만', 442),
 ('내', 439),
 ('때', 437),
 ('미국', 436),
 ('글로벌', 434),
 ('모두', 432),
 ('모든', 432),
 ('지역', 431),
 ('후', 428),
 ('등의', 428),
 ('신종', 426),
 ('코로나바이러스', 425),
 

In [14]:
from konlpy import init_jvm
from konlpy.tag import Mecab, Komoran,Kkma,Hannanum

# 토큰화 및 토큰화 과정에서 불용어를 제거하는 함수입니다.
def preprocessing(data,mode,stopwords):
  text_data = []
  mode = mode.lower()

  if mode == "mecab":
      tokenizer = Mecab()
  elif mode == 'komoran':
      tokenizer = Komoran()
  elif mode == 'kkma':
      tokenizer = Kkma()
  elif mode == 'hannanum':
      tokenizer = Hannanum()        

  for sentence in data:
    temp_data = []
    #- 토큰화
    temp_data = tokenizer.morphs(sentence) 
    #- 불용어 제거
    temp_data = [word for word in temp_data if not word in stopwords] 
    text_data.append(temp_data)

  text_data = list(map(' '.join, text_data))

  return text_data

In [15]:
def tfidf_vectorizer(data):
  data_counts = count_vect.transform(data)
  data_tfidf = tfidf_transformer.transform(data_counts)
  return data_tfidf

In [16]:
text_data1 = preprocessing(df['news'],"Komoran",stopwords)
# text_data2 = preprocessing(df['news'],"komoran",stopwords)
# text_data3 = preprocessing(df['news'],"kkma",stopwords)
# text_data4 = preprocessing(df['news'],"hannanum",stopwords)

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics


X_train, X_test, y_train, y_test = train_test_split(text_data1, df['code'], random_state = 0)
#- 단어의 수를 카운트하는 사이킷런의 카운트벡터라이저입니다.
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

#- 카운트벡터라이저의 결과로부터 TF-IDF 결과를 얻습니다.
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

#- 나이브 베이즈 분류기를 수행합니다.
#- X_train은 TF-IDF 벡터, y_train은 레이블입니다.
clf = MultinomialNB().fit(X_train_tfidf, y_train)
y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))


# X_train, X_test, y_train, y_test = train_test_split(text_data2, df['code'], random_state = 0)
# #- 단어의 수를 카운트하는 사이킷런의 카운트벡터라이저입니다.
# count_vect = CountVectorizer()
# X_train_counts = count_vect.fit_transform(X_train)

# #- 카운트벡터라이저의 결과로부터 TF-IDF 결과를 얻습니다.
# tfidf_transformer = TfidfTransformer()
# X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

# #- 나이브 베이즈 분류기를 수행합니다.
# #- X_train은 TF-IDF 벡터, y_train은 레이블입니다.
# clf = MultinomialNB().fit(X_train_tfidf, y_train)
# y_pred = clf.predict(tfidf_vectorizer(X_test))
# print(metrics.classification_report(y_test, y_pred))


# X_train, X_test, y_train, y_test = train_test_split(text_data3, df['code'], random_state = 0)
# #- 단어의 수를 카운트하는 사이킷런의 카운트벡터라이저입니다.
# count_vect = CountVectorizer()
# X_train_counts = count_vect.fit_transform(X_train)

# #- 카운트벡터라이저의 결과로부터 TF-IDF 결과를 얻습니다.
# tfidf_transformer = TfidfTransformer()
# X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

# #- 나이브 베이즈 분류기를 수행합니다.
# #- X_train은 TF-IDF 벡터, y_train은 레이블입니다.
# clf = MultinomialNB().fit(X_train_tfidf, y_train)
# y_pred = clf.predict(tfidf_vectorizer(X_test))
# print(metrics.classification_report(y_test, y_pred))


# X_train, X_test, y_train, y_test = train_test_split(text_data4, df['code'], random_state = 0)
# #- 단어의 수를 카운트하는 사이킷런의 카운트벡터라이저입니다.
# count_vect = CountVectorizer()
# X_train_counts = count_vect.fit_transform(X_train)

# #- 카운트벡터라이저의 결과로부터 TF-IDF 결과를 얻습니다.
# tfidf_transformer = TfidfTransformer()
# X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

# #- 나이브 베이즈 분류기를 수행합니다.
# #- X_train은 TF-IDF 벡터, y_train은 레이블입니다.
# clf = MultinomialNB().fit(X_train_tfidf, y_train)
# y_pred = clf.predict(tfidf_vectorizer(X_test))
# print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       IT/과학       0.68      0.86      0.76       203
          경제       0.76      0.85      0.80       235
          사회       0.87      0.73      0.79       235
       생활/문화       0.85      0.68      0.76       229

    accuracy                           0.78       902
   macro avg       0.79      0.78      0.78       902
weighted avg       0.79      0.78      0.78       902



In [18]:
leng = len(stopwords)
for duple in duple_dict:
    if not duple[0] in stopwords:
        stopwords.append(duple[0])
        leng+=1
        if leng==50:
            break
            
text_data1 = preprocessing(df['news'],"Komoran",stopwords)
print("mecab")
# text_data2 = preprocessing(df['news'],"komoran",stopwords)
# print("komoran")
# text_data3 = preprocessing(df['news'],"kkma",stopwords)
# print("kkma")
# text_data4 = preprocessing(df['news'],"hannanum",stopwords)
# print("hannanum")

mecab


In [19]:

X_train, X_test, y_train, y_test = train_test_split(text_data1, df['code'], random_state = 0)
#- 단어의 수를 카운트하는 사이킷런의 카운트벡터라이저입니다.
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

#- 카운트벡터라이저의 결과로부터 TF-IDF 결과를 얻습니다.
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

#- 나이브 베이즈 분류기를 수행합니다.
#- X_train은 TF-IDF 벡터, y_train은 레이블입니다.
clf = MultinomialNB().fit(X_train_tfidf, y_train)
y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))


# X_train, X_test, y_train, y_test = train_test_split(text_data2, df['code'], random_state = 0)
# #- 단어의 수를 카운트하는 사이킷런의 카운트벡터라이저입니다.
# count_vect = CountVectorizer()
# X_train_counts = count_vect.fit_transform(X_train)

# #- 카운트벡터라이저의 결과로부터 TF-IDF 결과를 얻습니다.
# tfidf_transformer = TfidfTransformer()
# X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

# #- 나이브 베이즈 분류기를 수행합니다.
# #- X_train은 TF-IDF 벡터, y_train은 레이블입니다.
# clf = MultinomialNB().fit(X_train_tfidf, y_train)
# y_pred = clf.predict(tfidf_vectorizer(X_test))
# print(metrics.classification_report(y_test, y_pred))


# X_train, X_test, y_train, y_test = train_test_split(text_data3, df['code'], random_state = 0)
# #- 단어의 수를 카운트하는 사이킷런의 카운트벡터라이저입니다.
# count_vect = CountVectorizer()
# X_train_counts = count_vect.fit_transform(X_train)

# #- 카운트벡터라이저의 결과로부터 TF-IDF 결과를 얻습니다.
# tfidf_transformer = TfidfTransformer()
# X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

# #- 나이브 베이즈 분류기를 수행합니다.
# #- X_train은 TF-IDF 벡터, y_train은 레이블입니다.
# clf = MultinomialNB().fit(X_train_tfidf, y_train)
# y_pred = clf.predict(tfidf_vectorizer(X_test))
# print(metrics.classification_report(y_test, y_pred))


# X_train, X_test, y_train, y_test = train_test_split(text_data4, df['code'], random_state = 0)
# #- 단어의 수를 카운트하는 사이킷런의 카운트벡터라이저입니다.
# count_vect = CountVectorizer()
# X_train_counts = count_vect.fit_transform(X_train)

# #- 카운트벡터라이저의 결과로부터 TF-IDF 결과를 얻습니다.
# tfidf_transformer = TfidfTransformer()
# X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

# #- 나이브 베이즈 분류기를 수행합니다.
# #- X_train은 TF-IDF 벡터, y_train은 레이블입니다.
# clf = MultinomialNB().fit(X_train_tfidf, y_train)
# y_pred = clf.predict(tfidf_vectorizer(X_test))
# print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       IT/과학       0.68      0.86      0.76       203
          경제       0.76      0.84      0.80       235
          사회       0.87      0.74      0.80       235
       생활/문화       0.85      0.68      0.76       229

    accuracy                           0.78       902
   macro avg       0.79      0.78      0.78       902
weighted avg       0.79      0.78      0.78       902



In [20]:
# for duple in duple_dict:
#     if not duple[0] in stopwords:
#         stopwords.append(duple[0])
#         leng+=1
#         if leng==500:
#             break

stopwords = ['에','는','은','을','했','에게',
             '있','이','의','하','한','다',
             '과','때문','할','수','무단',
             '따른','및','금지','전재',
             '경향신문','기자','는데','가',
             '등','들','파이낸셜','저작','등','뉴스',
            '된다','한다','뉴시스','연합뉴스','입니다','습니다','있다',
            '있는','사진','다고','다는','지만']
leng = len(stopwords)
for duple in duple_dict:
    if not duple[0] in stopwords and len(duple[0])==1:
        stopwords.append(duple[0])
        leng+=1
        if leng==50:
            break
print(stopwords)
# text_data2 = preprocessing(df['news'],"komoran",stopwords)
# text_data3 = preprocessing(df['news'],"kkma",stopwords)
# text_data4 = preprocessing(df['news'],"hannanum",stopwords)

['에', '는', '은', '을', '했', '에게', '있', '이', '의', '하', '한', '다', '과', '때문', '할', '수', '무단', '따른', '및', '금지', '전재', '경향신문', '기자', '는데', '가', '등', '들', '파이낸셜', '저작', '등', '뉴스', '된다', '한다', '뉴시스', '연합뉴스', '입니다', '습니다', '있다', '있는', '사진', '다고', '다는', '지만', '일', '년', '월', '전', '대', '개', '중']


In [21]:
text_data1 = preprocessing(df['news'],"Komoran",stopwords)

X_train, X_test, y_train, y_test = train_test_split(text_data1, df['code'], random_state = 0)
#- 단어의 수를 카운트하는 사이킷런의 카운트벡터라이저입니다.
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

#- 카운트벡터라이저의 결과로부터 TF-IDF 결과를 얻습니다.
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

#- 나이브 베이즈 분류기를 수행합니다.
#- X_train은 TF-IDF 벡터, y_train은 레이블입니다.
clf = MultinomialNB().fit(X_train_tfidf, y_train)
y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))


# X_train, X_test, y_train, y_test = train_test_split(text_data2, df['code'], random_state = 0)
# #- 단어의 수를 카운트하는 사이킷런의 카운트벡터라이저입니다.
# count_vect = CountVectorizer()
# X_train_counts = count_vect.fit_transform(X_train)

# #- 카운트벡터라이저의 결과로부터 TF-IDF 결과를 얻습니다.
# tfidf_transformer = TfidfTransformer()
# X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

# #- 나이브 베이즈 분류기를 수행합니다.
# #- X_train은 TF-IDF 벡터, y_train은 레이블입니다.
# clf = MultinomialNB().fit(X_train_tfidf, y_train)
# y_pred = clf.predict(tfidf_vectorizer(X_test))
# print(metrics.classification_report(y_test, y_pred))


# X_train, X_test, y_train, y_test = train_test_split(text_data3, df['code'], random_state = 0)
# #- 단어의 수를 카운트하는 사이킷런의 카운트벡터라이저입니다.
# count_vect = CountVectorizer()
# X_train_counts = count_vect.fit_transform(X_train)

# #- 카운트벡터라이저의 결과로부터 TF-IDF 결과를 얻습니다.
# tfidf_transformer = TfidfTransformer()
# X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

# #- 나이브 베이즈 분류기를 수행합니다.
# #- X_train은 TF-IDF 벡터, y_train은 레이블입니다.
# clf = MultinomialNB().fit(X_train_tfidf, y_train)
# y_pred = clf.predict(tfidf_vectorizer(X_test))
# print(metrics.classification_report(y_test, y_pred))


# X_train, X_test, y_train, y_test = train_test_split(text_data4, df['code'], random_state = 0)
# #- 단어의 수를 카운트하는 사이킷런의 카운트벡터라이저입니다.
# count_vect = CountVectorizer()
# X_train_counts = count_vect.fit_transform(X_train)

# #- 카운트벡터라이저의 결과로부터 TF-IDF 결과를 얻습니다.
# tfidf_transformer = TfidfTransformer()
# X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

# #- 나이브 베이즈 분류기를 수행합니다.
# #- X_train은 TF-IDF 벡터, y_train은 레이블입니다.
# clf = MultinomialNB().fit(X_train_tfidf, y_train)
# y_pred = clf.predict(tfidf_vectorizer(X_test))
# print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       IT/과학       0.68      0.86      0.76       203
          경제       0.76      0.85      0.80       235
          사회       0.87      0.73      0.79       235
       생활/문화       0.85      0.69      0.76       229

    accuracy                           0.78       902
   macro avg       0.79      0.78      0.78       902
weighted avg       0.79      0.78      0.78       902



In [22]:
stopwords = ['에','는','은','을','했','에게','있','이','의','하','한','다','과','때문','할','수','무단','따른','및','금지','전재','경향신문','기자','는데','가','등','들','파이낸셜','저작','등','뉴스','노컷뉴스','매일신문','그','나오','가지','씨','시키','만들','지금','생각하','그러','속','하나','집','살','모르','적','월','데','자신','안','어떤','내','경우','명','생각','시간','그녀','다시','이런','앞','보이','번','나','다른','어떻','여자','개','전','들','사실','이렇','점','싶','말','정도','좀','원','잘','통하','소리','놓','권','자','재','될','고','되','며','주','인','시','일','파','만','총','게','태','스','크','를','어','당','각','지난','관련','지만','사람','입니다','습니다','밝혔','지만','면서','지난','우리','이상','이용','경우','뉴시스','까지','다는','다면']

In [23]:
text_data1 = preprocessing(df['news'],"Komoran",stopwords)

X_train, X_test, y_train, y_test = train_test_split(text_data1, df['code'], random_state = 0)
#- 단어의 수를 카운트하는 사이킷런의 카운트벡터라이저입니다.
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

#- 카운트벡터라이저의 결과로부터 TF-IDF 결과를 얻습니다.
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

#- 나이브 베이즈 분류기를 수행합니다.
#- X_train은 TF-IDF 벡터, y_train은 레이블입니다.
clf = MultinomialNB().fit(X_train_tfidf, y_train)
y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       IT/과학       0.68      0.86      0.76       203
          경제       0.76      0.85      0.80       235
          사회       0.87      0.74      0.80       235
       생활/문화       0.85      0.69      0.76       229

    accuracy                           0.78       902
   macro avg       0.79      0.78      0.78       902
weighted avg       0.79      0.78      0.78       902



In [24]:
df=pd.concat([df1,df2],axis=0,ignore_index=True)

len1 = len(df.loc[df['code']=='사회'])
len2 = len(df.loc[df['code']=='경제'])
len3 = len(df.loc[df['code']=='생활/문화'])
len4 = len(df.loc[df['code']=='IT/과학'])
min_len = min(len1,len2,len3,len4)

rate1 = (len1-min_len)/len1
rate2 = (len2-min_len)/len2
rate3 = (len3-min_len)/len3
rate4 = (len4-min_len)/len4

df=df.drop(df.loc[df['code']=='사회'].sample(frac=rate1).index)
df=df.drop(df.loc[df['code']=='경제'].sample(frac=rate2).index)
df=df.drop(df.loc[df['code']=='생활/문화'].sample(frac=rate3).index)
df=df.drop(df.loc[df['code']=='IT/과학'].sample(frac=rate4).index)


In [25]:
stopwords = ['에','는','은','을','했','에게','있','이','의','하','한','다','과','때문','할','수','무단','따른','및','금지','전재','경향신문','기자','는데','가','등','들','파이낸셜','저작','등','뉴스','노컷뉴스','매일신문','그','나오','가지','씨','시키','만들','지금','생각하','그러','속','하나','집','살','모르','적','월','데','자신','안','어떤','내','경우','명','생각','시간','그녀','다시','이런','앞','보이','번','나','다른','어떻','여자','개','전','들','사실','이렇','점','싶','말','정도','좀','원','잘','통하','소리','놓','권','자','재','될','고','되','며','주','인','시','일','파','만','총','게','태','스','크','를','어','당','각','지난','관련','지만','사람','입니다','습니다','밝혔','지만','면서','지난','우리','이상','이용','경우','뉴시스','까지','다는','다면']

In [26]:
text_data1 = preprocessing(df['news'],"Komoran",stopwords)

X_train, X_test, y_train, y_test = train_test_split(text_data1, df['code'], random_state = 0)
#- 단어의 수를 카운트하는 사이킷런의 카운트벡터라이저입니다.
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

#- 카운트벡터라이저의 결과로부터 TF-IDF 결과를 얻습니다.
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

#- 나이브 베이즈 분류기를 수행합니다.
#- X_train은 TF-IDF 벡터, y_train은 레이블입니다.
clf = MultinomialNB().fit(X_train_tfidf, y_train)
y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       IT/과학       0.85      0.87      0.86       252
          경제       0.75      0.85      0.80       237
          사회       0.80      0.84      0.82       271
       생활/문화       0.85      0.68      0.76       240

    accuracy                           0.81      1000
   macro avg       0.81      0.81      0.81      1000
weighted avg       0.81      0.81      0.81      1000



In [27]:
df

,news,code
0,파주시청. 사진제공=파주시 파주시청. 사진제공=파주시\n\n[파주=파이낸셜뉴스 강근...,사회
1,동영상 뉴스\n\n이천 물류창고 화재 발화지점으로 지목된 지하 2층에서 산소절단기의...,사회
8,"6일 등교수업을 앞둔 경북 한 학교의 보건실에는 손소독제, 마스크 등 방역물품이 상...",사회
10,안병용 의정부시장 이천화재 희생자 합동분향소 헌화. 사진제공=의정부시 안병용 의정부...,사회
12,동영상 뉴스\n\n[앵커]다음 주 고등학교 3학년부터 순차적인 등교수업이 시작되는데...,사회
...,...,...
8818,삼성전자 중저가 스마트폰 갤럭시 A51. 삼성전자 제공 삼성전자 중저가 스마트폰 갤...,IT/과학
8819,신종 코로나바이러스 감염증(코로나19)으로 정부와 금융권의 저금리 대출 지원이 늘어...,IT/과학
8824,아마존 [AFP=연합뉴스 자료사진] 아마존 [AFP=연합뉴스 자료사진]\n\n직원들...,IT/과학
8825,"제21대 국회의원선거가 임박한 가운데, 투표 및 개표 방송을 준비하는 기업들의 움직...",IT/과학


In [29]:
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

text_clf = Pipeline([('clf', SGDClassifier(loss='hinge', penalty='l2',
                       alpha=1e-3, random_state=42,
                       max_iter=5, tol=None))
])
text_clf.fit(X_train_tfidf, y_train)
y_pred = text_clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       IT/과학       0.90      0.85      0.87       252
          경제       0.86      0.83      0.84       237
          사회       0.78      0.89      0.83       271
       생활/문화       0.86      0.80      0.83       240

    accuracy                           0.84      1000
   macro avg       0.85      0.84      0.85      1000
weighted avg       0.85      0.84      0.85      1000

